In [258]:
import cvxpy as cp
import numpy as np

N, M, L = 6, 12, 2
group_size = [int(np.ceil(M/2)), int(M-np.ceil(M/2))] 
temp_group_order = np.random.permutation(np.arange(M))
group_ids = (temp_group_order[:group_size[0]], temp_group_order[group_size[0]+1:])

def v_theta(theta):
    powers = np.arange(N)
    powers = np.expand_dims(powers, 1)
    theta1 = np.expand_dims(theta.copy(), 0)
    H = np.exp(1j*theta1*powers)
    return H

# create H
G = len(group_size)

# randomly generated phi    
phi = np.random.rand(M)*2*np.pi

#TODO: Generate structured Phi (users in the same group are close together)
phi = np.sort(phi)
nbeams = 2
perm = np.random.permutation(np.arange(6))
phi_new = np.array([])
for i in perm:
    phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
phi = phi_new.copy()

theta = -np.pi*np.sin(phi)
H = v_theta(theta)
gamma = 10

group_membership = np.zeros(M, dtype=np.int8)

for idx in range(len(group_ids)): 
    for i in group_ids[idx]:
        group_membership[i] = int(idx)
print(group_membership)
X = []
for i in range(G):
    X.append(cp.Variable((N,N), hermitian = True))
    
obj_val = 0
for k in range(G):
    obj_val += cp.real(cp.trace(X[k]))
obj = cp.Minimize(obj_val)
# obj = cp.Minimize(cp.trace(cp.real(np.sum(X, 0))))

constraints = []

for i in range(M):
    Qi = np.matmul(np.expand_dims(H[:,i],1), np.expand_dims(H[:,i],1).conj().T)
    k = group_membership[i]
    
    #     maskI = np.ones(G)
    #     maskI[k] = 0
    #     maskI = np.expand_dims(maskI, (1,2))
    mask_list = [i for i in range(G)]
    del mask_list[k]
    print(k, mask_list)
    constraints.append(cp.real(cp.trace(Qi @ X[k])) - gamma*cp.real(cp.sum([cp.trace(Qi @ X[l]) for l in mask_list])) >= gamma)
#     constraints
for k in range(G):
    constraints.append(X[k] >> 0)

prob = cp.Problem(obj, constraints)
prob.solve(verbose=False)

if prob.status is not 'infeasible':
    eigval = []
    eigvec = []
    for i in range(G):
        a,b = np.linalg.eig(X[i].value)
        eigval.append(np.real(a))
        eigvec.append(b)
        print(i, eigval[i][0]/np.sum(eigval[i]))
        assert eigval[i][0]/np.sum(eigval[i]) > 0.99, 'not rank 1 id {}, {}'.format(i, eigval[i])



[0 1 0 1 0 1 0 1 1 0 0 0]
0 [1]
1 [0]
0 [1]
1 [0]
0 [1]
1 [0]
0 [1]
1 [0]
1 [0]
0 [1]
0 [1]
0 [1]


SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [3]:
np.arange(-62,-25,4).shape

(10,)

In [4]:
group_ids

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29]))

In [16]:
H[:,21]

array([ 1.        +0.j        , -0.88820588+0.45944566j,
        0.57781937-0.81616467j, -0.13823925+0.99039886j,
       -0.33224955-0.94319152j,  0.72845125+0.68509764j])

In [20]:
np.sin(np.pi)

-2.4492935982947064e-16

In [15]:
np.exp(1j*(N-1)*theta[21])

(0.7284512491213102+0.6850976409633905j)

In [11]:
np.sin(theta[0])

-0.9813155276140235

In [21]:
i = 0
Qi = np.matmul(np.expand_dims(H[:,i],1), np.expand_dims(H[:,i],1).conj().T)


In [22]:
Qi

array([[ 1.        +0.j        ,  0.19240539+0.98131553j,
        -0.92596033+0.3776208j , -0.54872492-0.83600297j,
         0.71480506-0.69932376j,  0.82378961+0.56689565j],
       [ 0.19240539-0.98131553j,  1.        +0.j        ,
         0.19240539+0.98131553j, -0.92596033+0.3776208j ,
        -0.54872492-0.83600297j,  0.71480506-0.69932376j],
       [-0.92596033-0.3776208j ,  0.19240539-0.98131553j,
         1.        +0.j        ,  0.19240539+0.98131553j,
        -0.92596033+0.3776208j , -0.54872492-0.83600297j],
       [-0.54872492+0.83600297j, -0.92596033-0.3776208j ,
         0.19240539-0.98131553j,  1.        +0.j        ,
         0.19240539+0.98131553j, -0.92596033+0.3776208j ],
       [ 0.71480506+0.69932376j, -0.54872492+0.83600297j,
        -0.92596033-0.3776208j ,  0.19240539-0.98131553j,
         1.        +0.j        ,  0.19240539+0.98131553j],
       [ 0.82378961-0.56689565j,  0.71480506+0.69932376j,
        -0.54872492+0.83600297j, -0.92596033-0.3776208j ,
         

In [23]:
H[:,i]

array([ 1.        -0.j        ,  0.19240539-0.98131553j,
       -0.92596033-0.3776208j , -0.54872492+0.83600297j,
        0.71480506+0.69932376j,  0.82378961-0.56689565j])

In [29]:
H[1,0]*H[1,0].conj()

(1+0j)

In [30]:
H[1,0]

(0.19240539302111753-0.9813155276140235j)

In [31]:
group_membership

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2], dtype=int8)

In [25]:
H[0,1]

(1-0j)

In [20]:
import cvxpy as cp
import numpy as np

N, M= 6, 30
group_size = [10,10,10] 
temp_group_order = np.arange(M)
group_ids = (temp_group_order[:group_size[0]], temp_group_order[group_size[0]:group_size[0]+group_size[1]], temp_group_order[group_size[0]+group_size[1]:])

def v_theta(theta):
    powers = np.arange(N)
    powers = np.expand_dims(powers, 1)
    theta1 = np.expand_dims(theta.copy(), 0)
    H = np.exp(1j*theta1*powers)
    return H

# create H
G = len(group_size)

phi = np.concatenate((np.arange(26,63,4)*np.pi/180,
         np.arange(-18,19,4)*np.pi/180,
         np.arange(-62,-25,4)*np.pi/180)
        )

theta = -np.pi*np.sin(phi)
H = v_theta(theta)


group_membership = np.zeros(M, dtype=np.int8)

for idx in range(len(group_ids)): 
    for i in group_ids[idx]:
        group_membership[i] = int(idx)
print(group_membership)

ratios = []

for gamma in np.arange(10,21,2.5):
    
    X = []
    for k in range(G):
        X.append(cp.Variable((N,N), hermitian = True))
    s = cp.Variable(M, complex=False)

    obj_val = 0
    for k in range(G):
        obj_val += cp.real(cp.trace(X[k]))
    obj = cp.Minimize(obj_val)
    # obj = cp.Minimize(cp.trace(cp.real(np.sum(X, 0))))

    constraints = []

    for i in range(M):
        Qi = np.matmul(np.expand_dims(H[:,i],1), np.expand_dims(H[:,i],1).conj().T)
        k = group_membership[i]

        #     maskI = np.ones(G)
        #     maskI[k] = 0
        #     maskI = np.expand_dims(maskI, (1,2))
        mask_list = [i for i in range(G)]
        del mask_list[k]
        print(k, mask_list)
        constraints.append(cp.real(cp.trace(Qi @ X[k])) - gamma*cp.real(cp.sum([cp.trace(Qi @ X[l]) for l in mask_list])) - s[i]== gamma)
        constraints.append(s[i] >= 0)

    for k in range(G):
        constraints.append(X[k] >> 0)

    prob = cp.Problem(obj, constraints)
    prob.solve(verbose=False)

    if prob.status is not 'infeasible':
        eigval = []
        eigvec = []
        for i in range(G):
            a,b = np.linalg.eig(X[i].value)
            eigval.append(np.real(a))
            eigvec.append(b)
            print(i, eigval[i][0]/np.sum(eigval[i]), eigval[i])
    #         assert eigval[i][0]/np.sum(eigval[i]) > 0.99, 'not rank 1 id {}, {}'.format(i, eigval[i])
        ratios.append(eigval[1][0]/np.sum(eigval[1]))


[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
0 0.9999999987378122 [ 7.99346995e+00  9.72473221e-09  2.92725535e-09 -5.63017895e-10
 -1.11213731e-09 -8.87572713e-10]
1 0.7942484341584697 [ 9.79671885e+00  2.53785863e+00 -9.85921165e-10 -5.75840109e-10
  2.01370771e-09  2.37419228e-09]
2 0.9999999987378123 [ 7.99346995e+00  9.72473189e-09  2.92725501e-09 -5.63018111e-10
 -1.11213743e-09 -8.87572560e-10]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
0 [1, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
1 [0, 2]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
2 [0, 1]
0 0.9999999978268246 [ 9.44405808e+00 

In [21]:
ratios

[0.7942484341584697,
 0.9116370832414707,
 0.9513936883900957,
 0.9843512273251299,
 0.9999993324801575]

In [7]:
10*np.log10(20)

13.010299956639813

In [41]:
0.25/94.74

0.002638800928857927

In [42]:
25.4/52.06

0.48789857856319624

In [43]:
52.06*3.19

166.0714

In [45]:
25.70*3/2.75


28.036363636363635

In [358]:
H.shape

(6, 30)

In [359]:
phi.shape

(30,)

In [264]:

np.arange(26,63,4).shape

(10,)

In [256]:

N, M, L = 6, 12, 2
group_size = [int(np.ceil(M/3)), int(np.ceil(M/3)), M-2*int(np.ceil(M/3))] 
temp_group_order = np.arange(M)
group_ids = (temp_group_order[:group_size[0]], temp_group_order[group_size[0]:group_size[0]+group_size[1]], temp_group_order[group_size[0]+group_size[1]:])

def v_theta(theta):
    powers = np.arange(N)
    powers = np.expand_dims(powers, 1)
    theta1 = np.expand_dims(theta.copy(), 0)
    H = np.exp(1j*theta1*powers)
    return H

# create H
G = len(group_size)
# randomly generated phi    
phi = np.random.rand(M)*2*np.pi

#TODO: Generate structured Phi (users in the same group are close together)
phi = np.sort(phi)
nbeams = 2
perm = np.random.permutation(np.arange(6))
phi_new = np.array([])
for i in perm:
    phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))
phi = phi_new.copy()

theta = -np.pi*np.sin(phi)
H = v_theta(theta)
gamma = 10

group_membership = np.zeros(M, dtype=np.int8)

for idx in range(len(group_ids)): 
    for i in group_ids[idx]:
        group_membership[i] = int(idx)
print(group_membership)

[0 0 0 0 1 1 1 1 2 2 2 2]


In [257]:
group_ids

(array([0, 1, 2, 3]), array([4, 5, 6, 7]), array([ 8,  9, 10, 11]))

In [246]:
group_ids

(array([0, 1, 2, 3]),
 array([], dtype=int64),
 array([ 4,  5,  6,  7,  8,  9, 10, 11]))

In [245]:
group_membership

array([0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int8)

In [161]:
H[:,0]

array([ 1.        +0.j        , -0.59168744+0.80616746j,
       -0.29981195-0.95399832j,  0.94647737+0.32277018j,
       -0.82022559+0.57204019j,  0.02415699-0.99970818j])

In [162]:
phi[0]

3.9191461042893967

In [165]:
np.exp(1j*theta[0])

(-0.591687438390011+0.8061674610473105j)

In [166]:
theta

array([ 2.20394672,  2.37430313,  2.65641597,  2.6769027 ,  2.84387055,
        2.93584734,  2.81524503, -2.04492158, -1.84509409, -0.75299985,
       -0.71284169,  0.26163886,  1.51252961,  1.77127698, -0.38843414,
       -0.78523415, -0.88282811, -0.89062149, -2.17431212, -2.34595309,
       -2.43674027, -2.99946518, -2.94475207, -2.79188137, -2.63957869,
       -2.34556585, -2.2206742 , -2.10465683, -2.47598781, -2.54264511,
       -2.75917934, -2.9263942 , -3.14133256, -3.0868805 , -3.05525199,
        2.3331699 ,  2.3146145 ,  2.17612871,  2.13778864,  1.7725361 ,
        0.85389905,  0.3420472 ])

In [167]:
phi

array([3.9191461 , 3.99841292, 4.14921237, 4.16154789, 4.27351831,
       5.07630754, 5.17223618, 2.4327986 , 2.51385935, 2.89954891,
       2.9126945 , 3.22497146, 3.64390454, 3.74059037, 0.12395964,
       0.25262633, 0.28484939, 0.28743524, 0.76440138, 0.84314719,
       0.8877261 , 1.87274292, 1.92666643, 2.04712684, 2.14393336,
       2.29863077, 2.35653983, 2.40747401, 0.90776719, 0.94304299,
       1.07224058, 1.19851568, 1.55792834, 1.75769846, 1.80578529,
       5.44613504, 5.45491243, 5.51798251, 5.53476883, 5.68370223,
       6.00791767, 6.17409204])

In [146]:
X[0].value

In [140]:
phi

array([0.2508498 , 0.26803054, 0.35539567, 0.70886974, 0.83748051,
       0.87704437, 1.23559445, 1.35263569, 1.44059274, 1.75219669,
       1.93890498, 2.01271671, 2.11749125, 2.12096556, 2.12550443,
       2.13192347, 2.2164285 , 2.2711525 , 2.40321269, 2.6295079 ,
       2.75455518, 2.92890344, 3.13516732, 3.2566869 , 3.285362  ,
       3.35018562, 3.51276532, 3.77503588, 4.18994724, 4.26961919,
       4.60928697, 4.94470498, 5.29845141, 5.31228651, 5.63777788,
       5.84314746, 5.87869609, 5.9180385 , 6.01940733, 6.15219568,
       6.2198115 , 6.23104642])

In [150]:
phi = np.sort(phi)
nbeams = 6
perm = np.random.permutation(np.arange(6))
phi_new = np.array([])
for i in perm:
    phi_new = np.concatenate((phi_new, phi[int(i*M/nbeams):int((i+1)*M/nbeams)]))

In [151]:
perm

array([2, 4, 1, 5, 3, 0])

In [152]:
phi_new

array([2.07421981, 2.10872115, 2.22140963, 2.41182473, 2.63529269,
       2.64956995, 2.70227487, 3.99054704, 4.14429986, 4.32150717,
       4.48493765, 4.52710137, 4.56506545, 4.57821354, 1.31751019,
       1.58087793, 1.58661655, 1.60853961, 1.81031837, 2.05241805,
       2.05263843, 4.70403228, 5.00456998, 5.79671001, 5.87014325,
       5.89225127, 6.0917651 , 6.10909943, 2.89559257, 2.9074195 ,
       2.93043977, 3.65560747, 3.71308292, 3.897975  , 3.94512018,
       0.00749466, 0.26490646, 0.28110269, 0.57956424, 0.83981638,
       0.89659561, 1.10609623])

In [155]:
phi[28:35]

array([3.99054704, 4.14429986, 4.32150717, 4.48493765, 4.52710137,
       4.56506545, 4.57821354])

In [149]:
int((i+1)*M/nbeams)

21

In [147]:
M/nbeams

7.0

TypeError: ufunc 'right_shift' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [98]:
eigval[0]

array([ 1.15162796e+00,  2.72550202e-01, -1.42737724e-09,  5.91490062e-09])

In [100]:
eigvec

[array([[ 0.54962242+0.j        , -0.35020702-0.37291076j,
          0.53726138+0.j        , -0.04416286+0.38158571j],
        [-0.35463863+0.26860176j, -0.07589495-0.48221603j,
         -0.36208462-0.28327539j,  0.29693756+0.51407204j],
        [-0.08735831-0.43621534j,  0.40346574-0.27478702j,
          0.21043327+0.40873959j,  0.59366825+0.j        ],
        [ 0.41141706+0.36444538j,  0.51157381+0.j        ,
          0.10064257-0.52775075j,  0.3083354 -0.22910107j]])]

In [80]:
eigval[0]

array([ 9.97112202e-01,  1.96881254e-08, -8.41649643e-09, -1.20104409e-08])

In [83]:
eigval[0]/np.sum(eigval[0])

array([ 1.00000000e+00,  1.97451454e-08, -8.44087197e-09, -1.20452251e-08])

In [73]:
eig1

(array([ 2.11285507e+00-5.37938662e-17j, -2.96392900e-09-4.57805471e-18j,
        -2.99313843e-10+4.82779028e-17j,  2.30739606e-09+2.24250291e-17j]),
 array([[-0.56949132-0.22863755j,  0.60683342+0.j        ,
          0.30974868-0.32875541j, -0.11343511+0.19561654j],
        [-0.27478035+0.21886129j, -0.30933217-0.18991265j,
          0.54403644+0.j        , -0.28027281-0.60853414j],
        [ 0.17345554+0.30547943j,  0.00505276+0.36294309j,
          0.43943901-0.32073192j,  0.66997511+0.j        ],
        [ 0.61367377+0.j        ,  0.31026976-0.52151649j,
          0.44401075+0.08293868j, -0.1302236 +0.18486527j]]))

In [74]:
eig2

(array([ 2.34497201e+00+6.72081005e-18j,  1.18688441e-08-1.09092808e-16j,
        -5.36976326e-09+5.61445991e-18j, -3.15643514e-10+9.27157904e-17j]),
 array([[ 0.60678283-0.08628651j, -0.37733219-0.27328491j,
         -0.47374028+0.11848805j, -0.40710704+0.05567396j],
        [ 0.35248639-0.0110717j ,  0.49906539-0.18403671j,
          0.51140076+0.j        , -0.34324255+0.46189698j],
        [ 0.35053435-0.03866406j,  0.53191709+0.j        ,
         -0.37528835+0.34740376j,  0.57546874+0.j        ],
        [ 0.61288723+0.j        , -0.25947461+0.38695884j,
          0.4281425 -0.23486861j,  0.28750838-0.29355521j]]))

In [53]:
np.matmul(Qi, X[k])

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [55]:
X[k].shape

(4, 4)

In [52]:
mask_list

[1]

In [51]:
k

0

In [49]:
Qi.shape

(4, 4)

In [46]:
H[:,i].conj().T

AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [43]:
H.shape

(4, 6)

In [42]:
Qi.shape

()

In [ ]:
int

In [38]:
print(group_membership)

[1. 0. 0. 0. 0. 1.]


In [26]:
group_size

[3.0, 3.0]

In [16]:

def v_theta(theta):
    powers = np.arange(N)
    powers = np.expand_dims(powers, 1)
    theta1 = np.expand_dims(theta.copy(), 0)
    H = np.exp(1j*theta1*powers)
    return H

# create H
N, M, L = 4, 6, 2
# randomly generated phi    
phi = np.random.rand(M)*2*np.pi
lmbda = 0.3   # 1 GHz
d = lmbda/2
theta = -2*np.pi*d*np.sin(phi)/lmbda
H = v_theta(theta)


In [19]:
H

array([[ 1.        -0.j        ,  1.        -0.j        ,
         1.        -0.j        ,  1.        -0.j        ,
         1.        +0.j        ,  1.        +0.j        ],
       [-0.47198788-0.88160504j,  0.78478746-0.61976499j,
         0.59899059-0.80075606j, -0.12364439-0.99232659j,
         0.42051334+0.90728636j, -0.07762974+0.99698226j],
       [-0.55445488+0.83221379j,  0.23178272-0.97276758j,
        -0.28242055-0.95929069j, -0.96942413+0.24539124j,
        -0.64633706+0.76305203j, -0.98794725-0.15479094j],
       [ 0.99537985+0.09601539j, -0.42098711-0.90706662j,
        -0.93732509-0.34845613j,  0.36337211+0.93164409j,
        -0.96410005-0.26553924j,  0.23101791-0.9729495j ]])

In [23]:
np.exp(1j*theta[2]*3)

(-0.937325089853056-0.34845613200510706j)

In [11]:
a = np.arange(5)
a = np.expand_dims(a,0)
b = np.random.rand(5, 1)

In [12]:
b

array([[0.36999604],
       [0.67700636],
       [0.56716228],
       [0.61203048],
       [0.22310401]])

In [14]:
a

array([[0, 1, 2, 3, 4]])

In [13]:
a*b

array([[0.        , 0.36999604, 0.73999208, 1.10998812, 1.47998417],
       [0.        , 0.67700636, 1.35401273, 2.03101909, 2.70802546],
       [0.        , 0.56716228, 1.13432456, 1.70148684, 2.26864912],
       [0.        , 0.61203048, 1.22406096, 1.83609144, 2.44812192],
       [0.        , 0.22310401, 0.44620803, 0.66931204, 0.89241605]])

In [9]:
a*b

array([0.        , 0.54630904, 0.22795139, 0.6817552 , 2.6920472 ])

In [6]:
import numpy as np
np.sin(np.pi)

1.2246467991473532e-16

In [36]:
a = [i for i in range(5)]

In [41]:
a = a*4

In [42]:
a

[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]

In [43]:
del a[0]

In [44]:
a

[2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]

In [18]:
np.random.permutation(np.arange(5))

array([3, 1, 4, 0, 2])

In [19]:
import cvxpy as cp
import numpy as np
z = np.random.rand(2,4,4)
cp.trace(cpz, )

NameError: name 'cpz' is not defined

In [34]:
a = np.ones(2)
a[0] = 0

np.multiply(np.expand_dims(a,(1,2)),z)

array([[[0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.44206381, 0.00815696, 0.4613717 , 0.05246478],
        [0.21120387, 0.26819947, 0.5588794 , 0.57068144],
        [0.85635208, 0.39918318, 0.73618774, 0.04001974],
        [0.62588482, 0.67097912, 0.68529413, 0.14919735]]])

In [35]:
z

array([[[0.4970715 , 0.81639813, 0.42551751, 0.91564607],
        [0.11988634, 0.51029198, 0.31625184, 0.47981532],
        [0.05754733, 0.11599649, 0.25389323, 0.0292484 ],
        [0.85949664, 0.90251433, 0.42684592, 0.2232494 ]],

       [[0.44206381, 0.00815696, 0.4613717 , 0.05246478],
        [0.21120387, 0.26819947, 0.5588794 , 0.57068144],
        [0.85635208, 0.39918318, 0.73618774, 0.04001974],
        [0.62588482, 0.67097912, 0.68529413, 0.14919735]]])

In [30]:
np.expand_dims(a,(1,2)).shape

(2, 1, 1)

In [ ]:
a

In [14]:
cp.sum(z, 0).shape

ValueError: Expressions of dimension greater than 2 are not supported.